# Data Driven Websites with Flask


---

Let's take our weather API from last class and add a bit of complexity in the form of some formatting. In particular, we'll add some HTML (Hyper-Text Markup Language) – the standard markup language for any document meant to be displayed by a web browser. 

First things first, let's make sure that our app is up and running.

1. Navigate to desired directory
2. `mkdir library_app`
3. `cd library_app`
4. `touch library.py`

## `library.py`

```python 

from flask import Flask

app = Flask(__name__)

# begin our Horror API

@app.route('/horror')
def horror():

    import requests

    url = 'http://openlibrary.org/subjects/horror.json'

    parameters = {
        'published_in'   : 2000-2010
    }

    resp = requests.get(url, params=parameters)

    data = resp.json()

    titles = set()

    for i in range(0, len(data)):
        title = data['works'][i]['title']
        titles.add(title)

    return 'Titles: {}'.format(sorted(titles))

app.run(host='0.0.0.0', port=5000, debug=True) # anyone can connect, and we're running on port 5000

```

5. `python library.py`

If we go to http://0.0.0.0:5000/horror we'll see our app is up and running. 

---

So far so good. Let's get a bit fancier, though, and add a personalized welcome message to each user that visits the site. In library.py, let's add the following: 

## `library.py`

```python 

from datetime import * 

... # keep your /horror route here, but add the '/' below

@app.route("/")
def hello_visitor():
    datetime.now().strftime('%Y-%m-%d %H:%M:%S') # get the current time from datetime
    date = datetime.now().strftime('%Y-%m-%d') # get today's date from the current time 
    time = datetime.now().strftime('%H:%M:%S') # get the current time from the current time 
    message = "The date is {d} and the time is {t}" # create the message telling us the current date and time 
    return message.format(d=date, t=time) # return our message 

... # keep the 'app.run' info as well

```

Go back to http://0.0.0.0:5000/ and we'll see our message. 

---

## HTML

In order to prettify (add HTML) to our date and time message, we create a new HTML file in our directory. 

1. Navigate to our library_app directory
2. `mkdir templates`
3. `cd templates`
4. `touch index.html`

## `index.html`

```html 

<!DOCTYPE html>
<html>
<head>
    <title>Welcome Message</title>
</head>
<body>
    <div style="color: purple">
    <h1>Hello World! Here's some fun info! {{ message }}</h1>
        <strong>Below an image is displayed</strong>
        <img width="300" src="https://upload.wikimedia.org/wikipedia/en/thumb/5/58/NYU_logo.svg/1280px-NYU_logo.svg.png"/>
    </div>
</body>
</html> 

```

## `library.py`

```python 

from flask import render_template

# ...

@app.route("/")
def hello_visitor():
    datetime.now().strftime('%Y-%m-%d %H:%M:%S') # get the current time from datetime
    date = datetime.now().strftime('%Y-%m-%d') # get today's date from the current time
    time = datetime.now().strftime('%H:%M:%S') # get the current time from the current time
    message = "The date is {} and the time is {}".format(date, time) # create the message
    return render_template("index.html",message=message) # return our message

```

---

# Exercise 1

Looking back to last week, adjust your library Flask app so that rather than being greated by the date, time, and visitor count, a visitor is told their location, the weather, and temperature. 

In [1]:
# your code here

# Solution

## `library.py`

```python 

@app.route("/")
def hello_visitor(): 
    
    import requests
    
    geoip_url = 'http://api.ipstack.com/check?access_key=dd4cbbbe9d6b9f2709e5f0533644e547'
    resp = requests.get(geoip_url)
    data = resp.json()
    lon = data["longitude"]
    lat = data["latitude"]

    openweathermap_url = "http://api.openweathermap.org/data/2.5/weather"
    parameters = {
        'lat'   : str(lat),
        'lon'   : str(lon),
        'units' : 'imperial',
        'mode'  : 'json',
        'appid' : 'ffb7b9808e07c9135bdcc7d1e867253d'
    }

    resp = requests.get(openweathermap_url, params=parameters)
    data = resp.json()

    location = data['name']
    weather = data['weather'][0]['description']
    temperature = data['main']['temp']

    message = "Your Location: {}, Your Weather: {}, Your Temp: {}".format(location, weather, temperature)
    
    return render_template("index.html", message=message)

```

---

## Jinja Iterators

We can also leverage what is known as a "Jinja iterator" – basically allowing us to render if / else statements in our template. For example, let's alter index.html file:

## `index.html`

``` html

<!DOCTYPE html>
<html>
    
```

`{% if temperature <= 32 %}`

``` html

<div style = "color: blue">
<body>
  <h1>Hello! {{ message }}. It is below freezing outside, bring a jacket!</h1>
</body>
</div>

```

`{% endif %}`

`{% if temperature > 32 %}`

``` html

<div style = "color: red">
<body>
  <h1>Hello! {{ message }}. It is warm outside, awesome!</h1>
</body>
</div>

```

`{% endif %}`

```  
</html> 

```

## `library.py`

```python 

    # ... 
    
    location = data['name']
    weather = data['weather'][0]['description']
    temperature = data['main']['temp']

    message = "Your Location: {}, Your Weather: {}, Your Temp: {}".format(location, weather, temperature)
    
    return render_template("index.html", message=message, location=location, weather=weather, temperature=temperature)

```

---

# Exercise 2

Create a new file called horror.html that returns a blue list if there are fewer than 5 books returned and a red list if there are more than 5 books returned. Don't forget to adjust your /horror app route to call this new html file, too. 

In [ ]:
# your code here

# Solution

## `horror.html`

``` html

<!DOCTYPE html>
<html>
    
```

`{% if set_length < 5 %}`

``` html

<div style = "color: blue">
<body>
  <h1>{{ titles }}</h1>
</body>
</div>

```

`{% endif %}`

`{% if set_length > 5 %}`

``` html

<div style = "color: red">
<body>
  <h1>{{ titles }}</h1>
</body>
</div>

```

`{% endif %}`

```  
</html> 

```

## `library.py`

```python 

@app.route('/horror')
def horror():

    import requests

    url = 'http://openlibrary.org/subjects/horror.json'

    parameters = {
        'published_in'   : 2000-2010
    }

    resp = requests.get(url, params=parameters)

    data = resp.json()

    titles = set()

    for i in range(0, len(data)):
        title = data['works'][i]['title']
        titles.add(title)

    set_length = len(titles)

    return render_template('horror.html', titles=titles, set_length=set_length)

```

---

What happens if we want to call upon a SQL database, such as our Citibike database from Week 1, and display it in a Flask application?

## `library.py`

``` python
@app.route("/citibike")
def citibike():

    import sqlite3

    con = sqlite3.connect('/Users/siegmanA/Desktop/NYU_ProjectsinProgramming_Fall2020/Class9_Data_Driven_Websites/citibikeDataForFlask.db') # connect to our db
    cursor = con.cursor()

    # retrieve records of all tables and construct dictionaries for convenient usage at HTML tamplates

    station_data = cursor.execute("SELECT station_id, stationName, availableBikes, lastCommunicationTime FROM StationsData LIMIT 5")

    station_data = [{"station_id": i[0], "stationName": i[1], "availableBikes": i[2], "lastCommunicationTime": i[3]} for i in station_data]

    return render_template('citibike.html', station_data=station_data)
```

## `citibike.html` 
^ This sould be a new document in our `templates` folder

``` html
<!DOCTYPE html>
<html>
<body>
  <table>
    <thead>
      <tr>
        <th>Station ID</th>
        <th>Station Name</th>
        <th>Available Bikes</th>
        <th>Last Communication Time</th>
      </tr>
    </thead>
    <tbody>
      {% for i in station_data %}
	<tr>
        <td>{{ i.station_id }}</td>
        <td>{{ i.stationName }}</td>
        <td>{{ i.availableBikes }}</td>
        <td>{{ i.lastCommunicationTime }}</td>
	</tr>
      {% endfor %}
    </tbody> 
  </table>
</body>
</html>
```

---

## Getting Parameters and Passing to SQL

Now let's see how we can query for the status of a Citibike station over time.

Here, we want to create a new function, where we will pass the `station_id` as a **parameter**. Then our code will read the value of the parameter `station_id` and then will query the database to get the status of that station.

## `library.py`

``` python
from flask import request

@app.route("/station_status")
def station_status():

    import sqlite3

    user_station_id = request.args.get('station_id')

    con = sqlite3.connect('/Users/siegmanA/Desktop/NYU_ProjectsinProgramming_Fall2020/Class9_Data_Driven_Websites/citibikeDataForFlask.db') # connect to our db
    cursor = con.cursor()

    # retrieve records of all tables and construct dictionaries for convenient usage at HTML tamplates

    station_data = cursor.execute("SELECT station_id, stationName, availableBikes, lastCommunicationTime FROM StationsData WHERE station_id = ? LIMIT 5", [user_station_id,])

    station_data = [{"station_id": i[0], "stationName": i[1], "availableBikes": i[2], "lastCommunicationTime": i[3]} for i in station_data]

    return render_template('citibike.html', station_data=station_data,user_station_id=user_station_id)
```

Now we can pass a parameter through our URL, such as http://0.0.0.0:5000/station_status?station_id=281

---

## Bootstrap

Our HTML pages aren't exactly visually pleasing. Lucky for us, [Twitter's Bootstrap](http://getbootstrap.com/) is an excellent set of carefully crafted user interface elements, layouts and JavaScript tools, easily available to use in your web design project. 

All you need is to create a link to the [Bootstrap resources](http://getbootstrap.com/getting-started/#download-cdn) in the HTML head and use the predefined CSS styles.

### `templates/citibike.html` 

```html
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
</head>
<body>
    <div class="container">
        <div class="panel panel-info">
            <div class="panel-heading">
                <h1 class="panel-title">Citibike Stations</h1>
            </div>
            <table  class="table table-striped table-bordered table-hover">
    <thead>
      <tr>
        <th>Station ID</th>
        <th>Station Name</th>
        <th>Available Bikes</th>
        <th>Last Communication Time</th>
      </tr>
    </thead>
    <tbody>
      {% for i in station_data %}
	<tr>
        <td>{{ i.station_id }}</td>
        <td>{{ i.stationName }}</td>
        <td>{{ i.availableBikes }}</td>
        <td>{{ i.lastCommunicationTime }}</td>
	</tr>
      {% endfor %}
    </tbody>
    </table>
  </div>
</div>
</body>
</html>
```

---

## Forms

HTML forms allow us to get user entries and then use the submitted values as parameters in our code. (Check out [W3 Schools](https://www.w3schools.com/html/html_forms.asp) for more details.)

Imagine we want to search the Citibike stations by their name, and get back a list of the stations that contain that search string. A simple HTML form that can do that is listed below. 

Save the file  `search_stations.html` in your `templates` folder, and add a new route in library.py so that you can get back this page, when you visit the main page of your website.

## `search_stations.html`

```html
<!DOCTYPE html>
<html>
<head>
    <title>Search Citibike Stations</title>
</head>
<body>
      <form action="/search">
      Station Name:<br>
      <input type="text" name="name"><br>
    </form>
</body>
```

## `library.py`

```python
@app.route('/')
def home():
    return render_template('search_stations.html')
```

A key part of the form is the `action` script – it defines what URL we should go to when we submit the form. In our case, it will go to the `http://<your IP>:5000/search` with the parameter `name` having the value of the text that the user entered.

For example, if we enter `Bleecker` on the form, the form will direct us to the URL `http://<your IP>:5000/search?name=Bleecker`.

Now, we need to specify the corresponding route in our webserver (i.e., the `/search` route) and define what it will do. Add the below code to `library.py` to implement that route. 

## `library.py`

```python
@app.route('/search')
def search():

    name = request.args.get('name')

    con = sqlite3.connect('/Users/siegmanA/Desktop/library_app/citibikeDataForFlask.db') # connect to our db
    cursor = con.cursor()

    station_data = cursor.execute("SELECT station_id, stationName, availableBikes, lastCommunicationTime FROM StationsData WHERE stationName = ? LIMIT 5", [name,])

    station_data = [{"station_id": i[0], "stationName": i[1], "availableBikes": i[2], "lastCommunicationTime": i[3]} for i in station_data]

    return render_template('citibike.html', name=name, station_data=station_data)
```

---

# Exercise 3

Adjust the /search app route so that a user can search for stations based on the minimum number of available bikes at a given station. 

In [ ]:
# your code here

# Solution

## `library.py`

```python
@app.route('/search')
def search():

    name = request.args.get('name')

    con = sqlite3.connect('/Users/siegmanA/Desktop/library_app/citibikeDataForFlask.db') # connect to our db
    cursor = con.cursor()

    station_data = cursor.execute("SELECT station_id, stationName, availableBikes, lastCommunicationTime FROM StationsData WHERE availableBikes > ? LIMIT 5", [name,])

    station_data = [{"station_id": i[0], "stationName": i[1], "availableBikes": i[2], "lastCommunicationTime": i[3]} for i in station_data]

    return render_template('citibike.html', name=name, station_data=station_data)
```

## `search_stations.html`

``` html
<!DOCTYPE html>
<html>
<head>
    <title>Search Citibike Stations</title>
</head>
<body>
      <form action="/search">
      Minimum Number of Available Bikes:<br>
      <input type="number" name=name><br>
    </form>
</body>   
```